In [30]:
import math
import numpy as np
import pandas as pd
import sys
import re
import itertools
from functions import *
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import seaborn as sns
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
%matplotlib inline
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (10, 3)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data wrangling

#### Loaded dataframe with every actions for all students and dataframe with student pre/post scores

In [31]:
df.head(2)

,Activity,student,Time Stamp,Family,Family_tool,Family_default,Family_both,Action,Component,Outcome,...,current_#loops,current_#components,current_#battery,current_#circuitSwitch,current_#grabBagResistor,current_#lightBulb,current_#resistor,current_#seriesAmmeter,non_default_resistor_values,keep
0,a2,10009106,1363981168331,Construct,Construct,Construct,Construct,add,wire,None,...,0,0,0,0,0,0,0,0,0,True
1,a2,10009106,1363981175425,Construct,Construct,Construct,Construct,add,battery,None,...,0,0,0,0,0,0,0,0,0,True


In [32]:
df_scores.head(2)

,Wave,scaffolding,how many PhETs in the past?,how comfortable with phets?,p100,used this circuit sim before?,COMPLETEDNESS,PRIORKNOWLEDGE,COMPLEXITY,DIAGRAMS,...,Anon Student Id,incoming_knowledge,incoming_attitude,learning1score,learning1,learning2score,learning2,split pre,split post t2,three groups
0,March,not,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,high,low,NaN,low,NaN,low,high,high,HH
1,March,scaff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,high,low,NaN,low,NaN,low,high,high,HH


#### Crop activity data to get rid of activity #3

In [33]:
max_time = 25*60*1000
max_times = {s:min(df[df['student']==s]["Time Stamp"])+max_time for s in set(df['student'])}

def keep_by_time (row):
    if row["Time Stamp"] <= max_times[row["student"]]:
        return True
    else: 
        return False

df['keep'] = df.apply (lambda row: keep_by_time (row),axis=1)
df=df[df['keep']]

#### Define LL, LH, HL and HH groups based on median split on pre and bimodal dist of post tests

In [34]:
median_learning0 = 0.45
df_scores['split pre'] = df_scores.apply (lambda row: label_learning (median_learning0,row,"z pre"),axis=1)
median_learning2 = np.median(df_scores[df_scores['split pre']=='low']['z post t2'])
df_scores['split post t2'] = df_scores.apply (lambda row: label_learning (median_learning2,row,"z post t2"),axis=1)

In [35]:
for x in ['high','low']:
    for y in ['high','low']:
        print x,y, len(set(df_scores[(df_scores['split pre']==x)&(df_scores['split post t2']==y)]['student']))

high high 22
high low 3
low high 38
low low 36


In [36]:
def label_3_groups (row):
    if row['split pre']  == 'high' and row['split post t2'] == 'high':
        return 'HH'
    if row['split pre']  == 'high' and row['split post t2'] == 'low':
        return 'HL'
    if row['split pre']  == 'low' and row['split post t2'] == 'high':
        return 'LH'
    if row['split pre']  == 'low' and row['split post t2'] == 'low':
        return 'LL'
df_scores['three groups'] = df_scores.apply (lambda row: label_3_groups (row),axis=1)

#### Convert action families into string sequences

In [37]:
def convert(action, test_granular = True):
    ''' this function converts action families into single 
    codes more amenable for sequence interpretation'''

    if test_granular:
        converter =  {"Construct" : "C",
        "Interface" : "I",
        "Pause" : "P",
        "Test_basic" : "T1",
        "Test_complex" : "Tm",
        "Test_other" : "To",
        "Test_simple" : "T2",
        }
    else:
        converter =  {"Construct" : "C",
        "Interface" : "I",
        "Pause" : "P",
        "Test_basic" : "T",
        "Test_complex" : "T",
        "Test_other" : "T",
        "Test_simple" : "T",
        }   
    return converter[action]

In [129]:
''' 
this is a test cell to assess regex
'''
ignore=['I', 'To']
sequence = list(df[df['student']==3591123]["Family"])

#prints raw sequence
print ''.join([convert(action) for action in sequence if convert(action) not in ignore])
print '------'

#prints blocked action sequence
block = re.sub(r'([A-Z][12m]{0,3})\1+', 
               r'\1',
               ''.join([convert(action) for action in sequence if convert(action) not in ignore]))
print block
print '------'

#prints blocked action for all action families except tests
block2 = re.sub(r'([CPI])\1+', 
                r'\1',
                ''.join([convert(action) for action in sequence if convert(action) not in ignore]))
print block2

CCCCCCCCCCCCCCCCCCCCCCCT1T1T1T1CT1T1T1PCCPCCCCCCCCCCCCT1CCCCCCPCCCCCCCCCCCCCCCCCCCT2T2T2CCCCCCCCCCCCCCCTmTmTmTmTmTmCCCCCCCCCCCCCCCCCCCCCCCCTmTmTmTmTmTmTmTmPCCCCCCCCCCTmT1CCCCCCCCCCCCCCCCCCT2T2T2TmTmTmTmTmTmTmTmTmTmT2T2CCTmTmTmTmTmTmTmTmPTmTmTmTmTmTmTmTmTmTmTmTmTmTmTmPTmTmPCCCCCCCCCCCCCCCPT2T2PCCCCCCCCCCCCCCCCCCCCCCCCCCCCPCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCPCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCPCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCT1T1CCCCCCCCCCCCCCCCCC
------
CT1CT1PCPCT1CPCT2CTmCTmPCTmT1CT2TmT2CTmPTmPTmPCPT2PCPCPCPCT1C
------
CT1T1T1T1CT1T1T1PCPCT1CPCT2T2T2CTmTmTmTmTmTmCTmTmTmTmTmTmTmTmPCTmT1CT2T2T2TmTmTmTmTmTmTmTmTmTmT2T2CTmTmTmTmTmTmTmTmPTmTmTmTmTmTmTmTmTmTmTmTmTmTmTmPTmTmPCPT2T2PCPCPCPCT1T1C


In [122]:
'''
This creates dictionary (keys are student ID) and values are action sequences
'''
ignore=['I', 'To']
students = list(df['student'].unique())
sequences = {student:'' for student in students}
for student in students:
    sequence = list(df[df['student']==student]["Family"])
    sequences[student] = re.sub(r'([A-Z][12m]{0,3})\1+', 
                                r'\1',
                                ''.join([convert(action) for action in sequence 
                                         if convert(action) not in ignore]))

In [128]:
#testing regex for detecting number of instances a phrase appears in a student whole sequence
s = sequences[3591123]
# s = 'CTmCTmCTmCT2CT1CT1CT1P'
matches = re.finditer(r'(?=(T1))',s)
results = [match.group(1) for match in matches]
print len(results)

5


In [133]:
'''
This creates a df with all single actions counts for every students.
'''

phrases = 'C P T1 T2 Tm'.split()
data = {}
for phrase in phrases:
    data[phrase] = {}
    for student in students:
        rx = r'(?=({0}))'.format(phrase)
        matches = re.finditer(rx,sequences[student])
        data[phrase][student] = len([match.group(1) for match in matches]) 

dfcounts = pd.DataFrame(data = data)
dfcounts['T'] = dfcounts[['T1','T2','Tm']].sum(axis=1)
dfcounts['group'] = [list(df_scores[df_scores['student']==student]['three groups'])[0] for student in list(dfcounts.index)]
print dfcounts[dfcounts['group']=='HH']['T1'].tolist()
# dfcounts.to_csv('single_actions_counts_allBlocked.csv', sep=',', index=False)
                 

[6, 3, 5, 2, 1, 1, 9, 1, 1, 2, 1, 8, 5, 5, 2, 3, 5, 0, 5, 1, 2, 0]
